In [4]:
#This code is for adaptive GPU usage
import keras.backend as K
cfg = K.tf.ConfigProto()
cfg.gpu_options.allow_growth = True
K.set_session(K.tf.Session(config=cfg))

Using TensorFlow backend.


In [5]:
from keras.layers import LSTM, Dense, Input
from keras.models import Model
import keras.optimizers as O
import keras.losses as L
import keras.activations as A
from sklearn.metrics import accuracy_score
import numpy as np
from keras.utils import np_utils
from numpy.random import seed

seed(1)

In [6]:
def build_model(num_timesteps = 50, num_input = 9, num_hidden = 128, num_classes = 3):
    '''
        Function that builds and returns a basline LSTM model.
        Architecture : 
        Layer 1 : Input Layer.
        Layer 2 : LSTM Layer.
        Layer 3 : Dense layer with num_classes number of nodes.
        
        I/P:
            num_timesteps : no. of timesteps that are present in the dataset.
            num_input : no. of input signals for the Input layer.
            num_hidden : no. of hidden units for the LSTM layer.
            num_classes : no. of output classes, for the dense layer.
            
        Returns : 
            Constructed model.
    '''
    ip = Input(shape = (num_timesteps, num_input))
    x_ip = LSTM(num_hidden)(ip)
    x_op = Dense(num_classes, activation='softmax')(x_ip)
    
    model = Model(inputs = [ip], outputs = [x_op])
    
    model.summary()
    
    model.compile(optimizer = O.Adam(), loss = L.categorical_crossentropy, metrics = ['acc'])
    
    return model

In [7]:
def load_data(path = '/home/iot/Documents/dataset_fog_release/dataset/RAW/'):
    '''
        Function that takes in the path of a dataset and reads and returns the train, validation and test
        splits.
    '''
    x_train, y_train = np.load(path + 'x_train.npy'), np.load(path + 'y_train.npy')
    x_test, y_test = np.load(path + 'x_test.npy'), np.load(path + 'y_test.npy')
    x_val, y_val = np.load(path + 'x_val.npy'), np.load(path + 'y_val.npy')
    
    return x_train, y_train, x_test, y_test, x_val, y_val

In [10]:
path = "/home/iot/Documents/dataset_fog_release/dataset/RAW/"
x_train, y_train, x_test, y_test, x_val, y_val = load_data(path)

model = build_model(num_timesteps = x_train.shape[1], num_input = x_train.shape[-1], num_hidden = 128, num_classes = y_train.shape[-1])

model.fit(x_train,y_train,epochs=10,batch_size=32,validation_data=[x_val,y_val])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 50, 9)             0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 128)               70656     
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 387       
Total params: 71,043
Trainable params: 71,043
Non-trainable params: 0
_________________________________________________________________
Train on 526 samples, validate on 59 samples
Epoch 1/10
526/526 [==============================] - 1s 2ms/step - loss: 0.9179 - acc: 0.6255 - val_loss: 0.5936 - val_acc: 0.6780
Epoch 2/10
526/526 [==============================] - 1s 1ms/step - loss: 0.5657 - acc: 0.7510 - val_loss: 0.4538 - val_acc: 0.6949
Epoch 3/10
526/526 [==============================] - 1s 1ms/step - loss: 0.4355 - acc: 0.817

In [15]:
import time

start = time.time()

preds = model1.predict(x_test,batch_size=32)

end = time.time()
print(end - start)
#200ms vs #80ms

0.25051188468933105


In [7]:
#Convert the predictions into one-hot outputs.
preds = np_utils.to_categorical(np.argmax(preds,axis=1))

In [9]:
%%time
print ("Final Classification Accuracy : ",accuracy_score(y_test,preds))

Final Classification Accuracy :  0.8333333333333334
CPU times: user 1.47 ms, sys: 3.17 ms, total: 4.64 ms
Wall time: 11.1 ms


In [13]:
from keras.models import load_model

model.save('baseline_lstm.h5')  # creates a HDF5 file 'my_model.h5'
del model  # deletes the existing model

    # returns a compiled model
    # identical to the previous one


NameError: name 'model' is not defined

In [14]:
model1 = load_model('baseline_lstm.h5')